## Imports

In [3]:
import numpy as np

from src.lstm_model import LSTMModel
from src.imdb_data_loader import IMDBDataLoader
from src.load_data import prepare_data, load_vectors
from src.train_test import train, test, eval_epoch
from src.hyperparam_optimizers.grid_search import GridSearchOptimizer
from src.hyperparam_optimizers.random_search import RandomSearchOptimizer
from src.hyperparam_optimizers.hyperopt import HyperoptOptimizer
from src.hyperparam_optimizers.skopt import SkoptOptimizer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Hyperparams

In [4]:
params_grid = {
    'hidden_size': [64, 128, 256, 512],
    'num_layers': np.array([1, 2]),
    'dropout': np.array([0.5]),
    'bidirectional': np.array([True, False]),
    'batch_size': np.array([64, 256]),
    'lr': [1e-3, 1e-2, 1e-1]
}

## GridSearch

In [5]:
grid_search = GridSearchOptimizer()
grid_search.optimize('checkpoints', params_grid, i_epoch=20)
print(f'Best params: {grid_search.best_params}')
print(f'Best loss: {grid_search.best_loss}')
print(f'Best acc: {grid_search.best_accuracy}')
print(f'Total time: {grid_search.total_time}')
print(f'Number of iterations: {len(grid_search)}')

Grid search params: 100%|██████████| 96/96 [00:00<00:00, 5209.44it/s]

Best params: (512, 2, 0.5, True, 256, 0.001)
Best loss: 0.2502022793098372
Best acc: 0.90256
Total time: 135576.44630527496
Number of iterations: 96


## Random Search

In [6]:
random_search = RandomSearchOptimizer(96)
random_search.optimize('checkpoints', params_grid, i_epoch=20)
print(f'Best params: {random_search.best_params}')
print(f'Best loss: {random_search.best_loss}')
print(f'Best acc: {random_search.best_accuracy}')
print(f'Total time: {random_search.total_time}')
print(f'Number of iterations: {len(random_search)}')

Random search iters: 100%|██████████| 96/96 [00:00<00:00, 5097.13it/s]

Best params: (512, 2, 0.5, True, 256, 0.001)
Best loss: 0.2502022793098372
Best acc: 0.90256
Total time: 82030.34368491173
Number of iterations: 96


## Hyperopt

Использование `hyperopt` подразумевает следующие шаги:
- описание целевой функции для минимизации
- пространство, по которому будет осуществляться поиск
- база данных, в которой нужно хранить вычисления во всех точках поиска
- используемый алгоритм

Выбор поискового алгоритма осуществляется просто через параметр `algo=hyperopt.tpe.suggest`, `algo=hyperopt.random.suggest`

Так как вычисления будут проводиться на GPU, нет смысла распараллеливать оптимизацию гиперпараметров: всё может обвалиться по памяти.

In [7]:
hyperopt_search = HyperoptOptimizer(96)
hyperopt_search.optimize('checkpoints', params_grid, i_epoch=20)
print(f'Best params: {hyperopt_search.best_params}')
print(f'Best loss: {hyperopt_search.best_loss}')
print(f'Best acc: {hyperopt_search.best_accuracy}')
print(f'Total time: {hyperopt_search.total_time}')
print(f'Number of iterations: {len(hyperopt_search)}')

100%|██████████| 96/96 [00:00<00:00, 150.57trial/s, best loss: 0.2502022793098372]
Best params: [512, 2, 0.5, True, 256, 0.001]
Best loss: 0.2502022793098372
Best acc: 0.90256
Total time: 57540.012427806854
Number of iterations: 96


## Scikit-optimize

In [21]:
skopt_search = SkoptOptimizer(96)
skopt_search.optimize('checkpoints', params_grid, i_epoch=20)
print(f'Best params: {skopt_search.best_params}')
print(f'Best loss: {skopt_search.best_loss}')
print(f'Best acc: {skopt_search.best_accuracy}')
print(f'Total time: {skopt_search.total_time}')
print(f'Number of iterations: {len(skopt_search)}')

/home/local/AP-TEAM/ostroukhov/.pyenv/versions/habr/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/local/AP-TEAM/ostroukhov/.pyenv/versions/habr/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/local/AP-TEAM/ostroukhov/.pyenv/versions/habr/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/local/AP-TEAM/ostroukhov/.pyenv/versions/habr/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/local/AP-TEAM/ostroukhov/.pyenv/versions/habr/lib/python3.

Best params: [64, 2, 0.5, False, 64, 0.001]
Best loss: 0.2595431144966189
Best acc: 0.89656
Total time: 42971.08967876434
Number of iterations: 96
